# Multi-Agent Travel Planning Orchestrator System
 
## Overview
This notebook demonstrates the **orchestrator pattern** where a central agent delegates tasks to specialized worker agents. Unlike sequential chaining, the orchestrator intelligently routes each request to the appropriate specialist.

<div align="center">
<img src="lesson_3.png" alt="Alt text" width="750"/>
</div>
 
### Key Concepts Covered:
1. **Orchestrator Pattern**: Central agent routes requests to specialists
2. **Agent as Plugin**: Worker agents registered as plugins for the orchestrator
3. **Intelligent Delegation**: Orchestrator decides which agent handles each request
4. **ChatHistoryAgentThread**: Maintains conversation context across interactions
5. **Separation of Concerns**: Each agent has a narrow, focused responsibility

## 1. Setup and Configuration

In [1]:
import os
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_KEY")
url = os.getenv("URL")
api_version = "2024-12-01-preview"

## 2. Initialize Kernel
 
The kernel manages services and coordinates agent operations.

In [2]:
kernel = Kernel()

## 3. Configure Azure OpenAI Service
 
Create the chat completion service that will power all agents.

In [3]:
chat_service = AzureChatCompletion(
    deployment_name="none",  # Deployment name managed by base_url
    api_key=api_key,
    base_url=url,
    api_version=api_version  
)

# Register the service with the kernel
kernel.add_service(chat_service)

## 4. Create Specialized Worker Agents
 
These are **specialist agents** with narrow, focused expertise.
They will be used as **plugins** by the orchestrator agent.

### 4.1 Flight Booking Agent

In [4]:
flight_booking_agent = ChatCompletionAgent(
    service=chat_service,
    name="FlightBookingAgent",
    description="Handles flight bookings and airline-related queries.",
    instructions="""
    AI Agent Persona: Flight Booking Specialist
    Role: A specialized assistant focused exclusively on flight bookings and airline information.
    Behavior: Only handles flight-related queries. Does not answer questions outside this domain.
    Response Style: Provides clear, actionable flight booking guidance.
    
    Agent Instructions:
    - When asked about booking flights, provide step-by-step guidance on checking availability, 
      comparing prices, and booking tickets.
    - Mention key considerations: dates, destinations, layovers, airline preferences, budget.
    - Always suggest checking multiple booking platforms (Google Flights, Kayak, airline websites).
    - Do NOT respond to questions about hotels, car rentals, attractions, or other travel topics.
    - If the query is outside flight booking, respond: "I only handle flight bookings. Please contact another specialist."
    """
)

### 4.2 Hotel Accommodation Agent

In [5]:
hotel_accommodation_agent = ChatCompletionAgent(
    service=chat_service,
    name="HotelAccommodationAgent",
    description="Specialist for hotel and lodging recommendations.",
    instructions="""
    AI Agent Persona: Hotel Accommodation Specialist
    Role: A specialized assistant focused exclusively on hotel bookings and lodging.
    Behavior: Only handles accommodation-related queries. Does not answer questions outside this domain.
    Response Style: Provides detailed hotel booking recommendations.
    
    Agent Instructions:
    - When asked about hotels or accommodations, provide guidance on finding and booking hotels.
    - Mention key factors: location, price range, amenities, reviews, cancellation policies.
    - Suggest platforms like Booking.com, Hotels.com, Airbnb for different travel styles.
    - Recommend checking reviews on TripAdvisor before booking.
    - Do NOT respond to questions about flights, attractions, car rentals, or other travel topics.
    - If the query is outside hotel booking, respond: "I only handle hotel accommodations. Please contact another specialist."
    """
)

### 4.3 Local Attractions Agent

In [6]:
local_attractions_agent = ChatCompletionAgent(
    service=chat_service,
    name="LocalAttractionsAgent",
    description="Recommends local attractions, activities, and sightseeing options.",
    instructions="""
    AI Agent Persona: Local Attractions & Activities Specialist
    Role: A specialized assistant focused exclusively on tourist attractions and activities.
    Behavior: Only handles queries about things to do, attractions, and local experiences. 
    Response Style: Provides engaging recommendations for sightseeing and activities.
    
    Agent Instructions:
    - When asked about attractions or things to do, provide curated recommendations.
    - Categorize suggestions: historical sites, museums, outdoor activities, food experiences, nightlife.
    - Mention practical details: opening hours, ticket prices, best times to visit, booking requirements.
    - Suggest researching on TripAdvisor, GetYourGuide, or local tourism websites.
    - Do NOT respond to questions about flights, hotels, car rentals, or other travel logistics.
    - If the query is outside attractions/activities, respond: "I only handle local attractions and activities. Please contact another specialist."
    """
)

## 5. Create Orchestrator Agent
 
The **orchestrator** is the key innovation here. It:
- Does NOT answer questions directly
- Analyzes each request to determine the appropriate specialist
- Delegates to worker agents (registered as plugins)
- Routes requests intelligently based on query content

In [7]:
orchestrator_agent = ChatCompletionAgent(
    service=chat_service,
    name="TravelOrchestratorAgent",
    description="Routes travel-related queries to the appropriate specialist agent.",
    instructions="""
    AI Agent Persona: Travel Planning Coordinator
    Role: A management assistant that intelligently routes travel requests to specialized agents.
    Behavior: NEVER answers user questions directly. Always delegates to the appropriate specialist agent.
    Response Style: Routes requests by invoking the correct plugin/agent.
    
    Agent Instructions:
    DO NOT answer any questions directly yourself.
    
    Delegation Rules:
    If the request is about FLIGHTS, AIRLINES, or FLIGHT BOOKING → invoke FlightBookingAgent
    If the request is about HOTELS, ACCOMMODATIONS, or LODGING → invoke HotelAccommodationAgent
    If the request is about ATTRACTIONS, ACTIVITIES, or THINGS TO DO → invoke LocalAttractionsAgent
    
    If the request is about topics NONE of your specialists handle (e.g., travel insurance, car rentals, 
       visa requirements), respond: "I apologize, but I don't have a specialist for that type of request. 
       My team covers flights, hotels, and local attractions only."
    
    CRITICAL: You are a router, not an answerer. Always delegate to plugins.
    """
)

# Manual orchestration logic
async def route_query(prompt: str):
    # Step 1: Ask orchestrator to decide
    orchestrator_response = ""
    print("Invoking Orchestrator Agent...")
    async for message in orchestrator_agent.invoke(prompt, ):
        orchestrator_response += str(message)
    print(f"Orchestrator Response: {orchestrator_response}")

    # Step 2: Decide which agent to call based on orchestrator's response
    if "flight booking" in orchestrator_response.lower() or "FlightBookingAgent" in orchestrator_response:
        print("Routing to FlightBookingAgent")
        agent = flight_booking_agent
    elif "hotel accommodation" in orchestrator_response.lower() or "HotelAccommodationAgent" in orchestrator_response:
        print("Routing to HotelAccommodationAgent")
        agent = hotel_accommodation_agent
    elif "local attractions" in orchestrator_response.lower() or "LocalAttractionsAgent" in orchestrator_response:
        print("Routing to LocalAttractionsAgent")
        agent = local_attractions_agent
    else:
        return orchestrator_response  # Out-of-scope fallback

    # Step 3: Invoke the selected agent
    final_response = ""
    async for message in agent.invoke(prompt, ):
        final_response += "Agent Responding: " + agent.name + "\n" + str(message)

    return final_response

## 6. Test the Orchestrator with Different Queries
 
We'll test various travel-related queries to see how the orchestrator routes them.

In [8]:
async def main():

    prompts = [
        "How do I book a flight to Paris?",
        "What are the best hotels in Tokyo?",
        "What attractions should I visit in Rome?",
        "How do I rent a car for my trip?",
        "I need travel insurance recommendations"
    ]

    for i, prompt in enumerate(prompts, start=1):
        print(f"\n{'='*60}\nQuery #{i}: {prompt}\n{'='*60}")
        response = await route_query(prompt)
        print(f"\nFinal Response:\n{response}")


## 7. Run the Demo

In [9]:
await main()


Query #1: How do I book a flight to Paris?
Invoking Orchestrator Agent...
Orchestrator Response: Routing your request to FlightBookingAgent for assistance with booking a flight to Paris.
Routing to FlightBookingAgent

Final Response:
Agent Responding: FlightBookingAgent
To book a flight to Paris, follow these steps:

1. **Decide on Details**:  
   - **Departure city**  
   - **Travel dates** (departure and return)  
   - **Number of passengers**  
   - **Class of service** (economy, business, etc.)  
   - **Any airline preferences**

2. **Compare Flight Options**:  
   - Go to flight comparison websites like **Google Flights**, **Kayak**, or **Skyscanner**.  
   - Enter your details (origin, destination: Paris, dates, number of travelers, class).  
   - Review options for direct and layover flights.  
   - Filter by flight times, airlines, price, or layover duration.

3. **Check Airline Websites**:  
   - Visit major airline websites (Air France, Delta, American Airlines, etc.) to com

# Using Semantic Kernel Orchestration

## 1. Setup and Configuration

In [10]:
import os
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent

# Load environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_KEY")
url = os.getenv("URL")
api_version = "2024-12-01-preview"

## 2. Instantiate Kernel

In [11]:
kernel = Kernel()

## 3. Instantiate a Chat Service and add to Kernel

In [12]:
chat_service = AzureChatCompletion(
    deployment_name="none",  # Deployment name managed by base_url
    api_key=api_key,
    base_url=url,
    api_version=api_version  
)

# Register the service with the kernel
kernel.add_service(chat_service)

## 4. Define Agents

In [13]:
hotel_agent = ChatCompletionAgent(
    service=chat_service,
    name="HotelAccommodationAgent",
    description="Handles hotel-related inquiries and bookings.",
    instructions="""
    AI Agent Persona: Hotel Booking Specialist
    Role: A dedicated agent for handling hotel inquiries and reservations.
    Behavior: Always responds with hotels recommendations for cheap motels. Never say no hotels available.
    Response Style: Friendly, informative, and focused on lodging details. Always include: Brought to you by HotelAccommodationAgent.
    """
)

flight_agent = ChatCompletionAgent(
    service=chat_service,
    name="FlightBookingAgent",
    description="Handles flight search, booking, and airline-related questions.",
    instructions="""
    AI Agent Persona: Flight Booking Assistant
    Role: A specialist in airline travel, flight schedules, and ticket reservations.
    Behavior: Always responds with flight recommendations on budget airlines. If flight details are missing, make your best guess based on user preferences.
    Response Format: 
    Input: User's flight inquiry.
    Output: Clear, concise flight booking guidance. 
    Always include the phrase: Brought to you by FlightBookingAgent.
    """
)

attractions_agent = ChatCompletionAgent(
    service=chat_service,
    name="LocalAttractionsAgent",
    description="Recommends local attractions and activities.",
    instructions="""
    AI Agent Persona: Local Experience Curator
    Role: A guide for discovering attractions, tours, and activities in a given destination.
    Behavior: Always responds with suggestions for historical sites and museums.
    Response Style: Enthusiastic, engaging, and tailored to traveler interests. Always include: brought to you by LocalAttractionsAgent.
    """
)

## 5. Define the Orchestrator Agent

In [14]:
orchestrator_agent = ChatCompletionAgent(
    service=chat_service,
    name="Orchestrator",
    instructions="""You are the routing orchestrator for a travel assistant system.

Your job is to decide which specialized agent should handle the user's request.

Available agents:
- HotelAccommodationAgent: Handles hotel-related inquiries and bookings.
- FlightBookingAgent: Handles flight search, booking, and airline-related questions.
- LocalAttractionsAgent: Recommends local attractions and activities.

If none of the agents can handle the request, respond with: "I apologize, but I don't have a specialist for that type of request."
"""
)

## 6. Define Orchestration Agents

In [15]:
from semantic_kernel.agents import OrchestrationHandoffs

handoffs = (
    OrchestrationHandoffs()
    .add_many(    # Use add_many to add multiple handoffs to the same source agent at once
        source_agent=orchestrator_agent.name,
        target_agents={
            hotel_agent.name: "Transfer to this agent if the issue is hotel related",
            flight_agent.name: "Transfer to this agent if the issue is flight related",
            attractions_agent.name: "Transfer to this agent if the issue is attractions related",
        },
    )
)

## 7. Define a Callback Function to see Agent messages

In [16]:
from semantic_kernel.contents import ChatMessageContent

def agent_response_callback(message: ChatMessageContent) -> None:
    if message.content.strip():
        print(f"{message.name}: {message.content}")
    else:
        print(f"{message.name}: [no content]")

## 8. Define the Orchestration

In [17]:
from semantic_kernel.agents import HandoffOrchestration

handoff_orchestration = HandoffOrchestration(
    members=[
        orchestrator_agent,
        hotel_agent,
        flight_agent,
        attractions_agent,
    ],
    handoffs=handoffs,
    agent_response_callback=agent_response_callback
)

## 7. Instatiate and start the Runtime

In [18]:
from semantic_kernel.agents.runtime import InProcessRuntime

runtime = InProcessRuntime()
runtime.start()

## 8. Run the Demo

In [19]:
orchestration_result = await handoff_orchestration.invoke(
    task="A customer is on the line asking for a hotel in Houston on Friday evening.",
    runtime=runtime,
)

value = await orchestration_result.get()
print(value)

orchestration_result = await handoff_orchestration.invoke(
    task="A customer is on the line asking for flights from Dallas to Houston on Friday evening.",
    runtime=runtime,
)

value = await orchestration_result.get()
print(value)

orchestration_result = await handoff_orchestration.invoke(
    task="A customer is on the line asking for attractions in Houston on Friday evening.",
    runtime=runtime,
)

value = await orchestration_result.get()
print(value)

orchestration_result = await handoff_orchestration.invoke(
    task="A customer is on the line asking for car rentals in Houston on Friday evening.",
    runtime=runtime,
)

value = await orchestration_result.get()
print(value)

Orchestrator: [no content]
Orchestrator: [no content]
Orchestrator: [no content]
HotelAccommodationAgent: Thank you for letting me know! I have some affordable motel recommendations in Houston for Friday evening:

1. Red Roof Inn Houston – Affordable rates, free WiFi, and located near downtown for easy access.
2. Motel 6 Houston Downtown – Budget-friendly, clean rooms, and complimentary morning coffee.
3. Super 8 by Wyndham Houston – Great deals, free breakfast, and convenient to local attractions.
4. Days Inn Houston – Low prices, outdoor pool, and close to dining options.

Would you like more details on any of these motels, or prefer to book a room now? Please let me know the customer's preferences or budget to help narrow down the choices.

Brought to you by HotelAccommodationAgent.
Task is completed with summary: No handoff agent name provided and no human response function set. Ending task.
Orchestrator: [no content]
Orchestrator: [no content]
Orchestrator: [no content]
FlightBook